In [58]:
import os
import wfdb
import pickle
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline

In [59]:
data_root = "./data"

In [60]:
with open("./dataset_RRI/07879.pickle","rb") as f:
    dataset = pickle.load(f)
X = dataset["X"]
y = dataset["y"]
start = dataset["start"]
end = dataset["end"]

In [61]:
len(start)

1

In [62]:
len(end)

1

In [63]:
len(X)

1

In [64]:
len(y)

1

In [51]:
for i in range(5):
    file = os.path.join("./dataset_RRI","dataset-cross"+str(i)+".pickle")
    with open(file,"rb") as f:
        dataset = pickle.load(f)
    print(dataset["test_key"])

['06453', '07162', '05091']
['08219', '04126', '06995']
['04126', '07162', '08434']
['08219', '07162', '04048']
['06426', '06995', '05261']


In [52]:
for i in range(5):
    file = os.path.join("./dataset_RRI/temp","dataset-cross"+str(i)+".pickle")
    with open(file,"rb") as f:
        dataset = pickle.load(f)
    print(dataset["test_key"])

FileNotFoundError: [Errno 2] No such file or directory: './dataset_RRI/temp/dataset-cross0.pickle'